In [ ]:
import numpy as np
import math
from scipy.linalg import svd
import cv2 as cv
import glob
from matplotlib import pyplot as plt

In [ ]:
### Calulating Camera Matrix

CHECKERBOARD = (8, 6)   # Defining the dimensions of checkerboard
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objpoints = []   # Creating vector to store vectors of 3D points for each checkerboard image
imgpoints = []   # Creating vector to store vectors of 2D points for each checkerboard image


# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0, :, :2] = np.mgrid[0:CHECKERBOARD[0],
                          0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

In [ ]:
images = glob.glob('/home/rushi/College/Fall_2022/Perception/HW2/Task3/data/*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv.findChessboardCorners(
        gray, CHECKERBOARD, cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_FAST_CHECK + cv.CALIB_CB_NORMALIZE_IMAGE)

    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv.cornerSubPix(
            gray, corners, (11, 11), (-1, -1), criteria)

        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)

cv.destroyAllWindows()

h, w = img.shape[:2]

ret, K, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [ ]:
#Printing Camera Matrix

print("Camera matrix : \n")
print(K)
print("dist : \n")
print(dist)

In [ ]:
def homogeneous_coord(pts):
    points = []
    for i in range(len(pts)):
        points.append([pts[i][0], pts[i][1]])

    # Convert to homogenous coordinates
    return np.concatenate((points, np.ones((len(points), 1))), axis=1)

def normalize(pts):
    mean = np.mean(pts, axis=0)
    centered = pts - mean * np.ones(pts.shape)
    squared = centered * centered
    distance = np.sqrt(squared[:, 0] + squared[:, 1])
    std = np.mean(distance)
    Tr1 = np.array([[1, 0, -mean[0]], [0, 1, -mean[1]], [0, 0, 1]])
    Tr2 = np.array([[math.sqrt(2) / std, 0, 0],
                   [0, math.sqrt(2) / std, 0], [0, 0, 1]])
    Tr = Tr2 @ Tr1
    return ((Tr @ pts.T).T, Tr)

def estimate_fundamental_matrix(p1, p2):
    # Linear Solution
    A_list = []
    for i in range(p1.shape[0]):
        u1 = p1[i][0]
        v1 = p1[i][1]
        u2 = p2[i][0]
        v2 = p2[i][1]

        # u1u1' v1u1' u1' u1v1' v1v1' v1' u1 v1 1
        # u1u2  v1u2  u2  u1v2  v1v2  v2  u1 v1 1
        A_list.append([u1*u2, v1*u2, u2, u1*v2, v1*v2, v2, u1, v1, 1])
    A = np.array(A_list)
    _, _, Vt = svd(A, full_matrices=False, overwrite_a=True)
    f = Vt[-1].reshape(3, 3)

    # Enforce 2-rank constraint
    s, v, d = svd(f, full_matrices=False, overwrite_a=True)
    f_hat = s @ np.diag([*v[:2], 0]) @ d
    print("Determinant of f_hat is: ", np.linalg.det(f_hat))
    return f_hat

In [ ]:
##    Drawing the epilines

def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = img1.shape
    img1 = cv.cvtColor(img1,cv.COLOR_GRAY2BGR)
    img2 = cv.cvtColor(img2,cv.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

img1 = cv.imread('/home/rushi/College/Fall_2022/Perception/HW2/Task3/left.jpg',0) ## Left Image
img2 = cv.imread('/home/rushi/College/Fall_2022/Perception/HW2/Task3/right.jpg',0)  ## Right image

sift = cv.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)
pts1 = []
pts2 = []

In [ ]:
## Getting the Fundamental Matrix

# ratio test using Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)

pts1 = np.int32(pts1)
pts2 = np.int32(pts2)

pt1_homo = homogeneous_coord(pts1)
pt2_homo = homogeneous_coord(pts2)


p1, Tr_p1 = normalize(pt1_homo)
p2, Tr_p2 = normalize(pt2_homo)

f_hat = estimate_fundamental_matrix(p1, p2)
F = Tr_p2.T @ f_hat @ Tr_p1

print("Fundamental Matrix is: ", F) 

In [ ]:
# Finding epilines corresponding to points in right image (second image) and drawing its lines on left image
lines1 = cv.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1,img2,lines1,pts1,pts2)

# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2,img1,lines2,pts2,pts1)
plt.subplot(121),plt.title('left_result'),plt.imshow(img5)
plt.subplot(122),plt.title('right_result'),plt.imshow(img3)
plt.show()

In [ ]:
#### Equation for calculationg the fundamental matrix
essential_mtx = K.T @ F @ K
print("Essential Matrix: \n\n", essential_mtx)

In [ ]:
## Decomposing the Essential matrix

U, S, Vt = svd(essential_mtx)
t = U[:,-1]
W = np.array([[0, -1.0, 0],
            [1.0, 0, 0],
            [0, 0, 1.0]])
R1 = -np.dot(U, np.dot(W, Vt))
R2 = -np.dot(U, np.dot(W.T, Vt))

### USE THESE IF YOUR PYTHON VERSION IS >=3.9
# R1 = np.dot(U, np.dot(W, Vt))
# R2 = np.dot(U, np.dot(W.T, Vt))

print("R1 = \n", R1, "\n\n R2 = \n", R2, "\n\n t = \n", t)